In [54]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

from data_processing.creator import Creator

from data_processing.readers.economic_data.eurostat_reader import EuroStatReader
from data_processing.readers.economic_data.exchange_rates import WorldBankRatesReader
from data_processing.readers.economic_data.oecd_economic_data import OECDEconData
from data_processing.readers.economic_data.policy_rates import PolicyRatesReader
from data_processing.readers.economic_data.world_bank_reader import WorldBankReader

from data_processing.readers.io_tables.icio_reader import ICIOReader
from data_processing.readers.io_tables.wiod_reader import WIODReader

from data_processing.readers.population_data.employment_data import WorldBankEmployment
from data_processing.readers.population_data.hfcs_reader import HFCSReader

from data_processing.readers.socioeconomic_data.wiod_sea_data import WIODSEAReader
from data_processing.util.download import download_data

In [55]:
# download_data(Path("raw_data"))

In [56]:
years = np.arange(2000,2022)
countries = [
     "GBR", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", 
    "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", 
    "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE", 
    "ZAF", "USA", "GBR", "BRA"
]

eurostat = [
    "nonfin_firm_debt_ratios",
    "get_total_nonfin_firm_debt",
    "get_total_fin_firm_debt",
    "nonfin_firm_deposit_ratios",

    # "get_quarterly_gdp",
    # "get_monthly_gdp",

    "get_total_nonfin_firm_deposits",
    "get_total_bank_equity",
    "cb_debt_ratios",
    "cb_equity_ratios",
    "general_gov_debt_ratios",
    "central_gov_debt_ratios",
    
    # shortterm_interest_rates

    "longterm_central_gov_bond_rates",
    "dividend_payout_ratio",
    "firm_risk_premium",
    "number_of_households",
    "taxrate_on_capital_formation"

    # "get_perc_sectoral_growth"
]

wb_exchange_rates = [
    "to_usd"
]

oecd = [
    "employees_by_industry",
    # "read_business_demography",
    "read_firm_size_zetas",
    "read_tau_sif",
    "read_tau_siw",
    "read_tau_firm",
    "read_tau_income",
    "read_short_term_interest_rates",
    "read_long_term_interest_rates",
    "read_tierone_reserves",
    "read_number_of_banks",
    "read_number_of_bank_branches",
    "read_number_of_bank_employees",
    "read_bank_distributed_profit",
    "read_bank_retained_profit",
    "read_bank_total_assets",
    "unemployment_benefits_gdp_pct",
    "read_bank_distributed_profit",
    "read_bank_retained_profit",
    "read_bank_total_assets",
    "unemployment_benefits_gdp_pct",
    "all_cash_benefits_gdp_pct",
    "general_gov_debt",
    # "get_unemployment_benefits"
    # "get_consumption_rates_by_income"
]

bis = [
    "cb_policy_rate"
]

wb = [
    "get_unemployment_rate",
    "get_participation_rate",
    "get_tau_vat",
    "get_tau_exp",
    "get_gini_coef",
    "get_historic_gdp",
    "get_current_monthly_gdp",
    # "get_log_inflation"
]

In [57]:
eurostat_reader = EuroStatReader(Path('raw_data/eurostat'), Path('raw_data/notation/wikipedia-iso-country-codes.csv'))
wbr_reader = WorldBankRatesReader(pd.read_csv('raw_data/exchange_rates/exchange_rates.csv', index_col=0))
oecd_reader = OECDEconData(Path("raw_data/oecd_econ"), "raw_data/icio/mappings.json","raw_data/icio/mappings.json", 100)
bis_reader = PolicyRatesReader(Path('raw_data/policy_rates/bis_cb_policy_rates.csv'), Path('raw_data/notation/wikipedia-iso-country-codes.csv'))
wb_reader = WorldBankReader(Path('raw_data/world_bank'))

def handle_tables(param, countries, years, reader):

    matrix = np.zeros((len(countries), len(years)))
    for c_i, c in enumerate(countries): 
        for y_i, y in enumerate(years):

            if param == "shortterm_gov_bond_rates":
                method = getattr(type(reader), param)
                try:
                    val = method(reader, country=c, year=y, months=0)
                except:
                    val = None

            elif param in ["quarterly_gdp", "quarterly_inflation"]: 
                method = getattr(type(reader), param)
                try:
                    val = method(reader, country=c, year=y, quarter=1)
                except:
                    val = None

            elif param in ["unemployment_rates","participation_rates"]:
                method=getattr(type(reader), param)
                try:
                    val = method(reader, year=y)[c]
                except:
                    val = None

            # elif param in  ["get_historic_gdp","get_historic_cpi"]:
            #     method=getattr(type(reader), param)
            #     try:
            #         val = method(reader, country=c)[y]
            #     except:
            #         val = None

            else:
                method = getattr(type(reader), param)
                try:
                    val = method(reader, country=c, year=y)
                    if not isinstance(val, (int, float, np.floating, np.integer)): val = 1
                except:
                    val = None
            
            try:
                matrix[c_i, y_i] = val
            except:
                print(val)
                print(param)
                raise Error

    return matrix.tolist()

def get_eurostat(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_eurostat"] = handle_tables(p, countries, years, eurostat_reader)

    return params

def get_wb_exchange_rates(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_worldbank"] = handle_tables(p, countries, years, wbr_reader)

    return params

def get_oecd(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_oecd"] = handle_tables(p, countries, years, oecd_reader)

    return params


def get_bis(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_bis"] = handle_tables(p, countries, years, bis_reader)

    return params

def get_wb(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_worldbank"] = handle_tables(p, countries, years, wb_reader)

    return params

In [58]:
wb_reader.get_unemployment_rate("GBR", 2020)

0.0447200012207031

In [59]:
c = countries
y = years

eurostat_results = get_eurostat(eurostat, c, y)
print("1")
wb_er_results = get_wb_exchange_rates(wb_exchange_rates, c, y)
print("2")
oecd_results = get_oecd(oecd, c, y)
print("3")
bis_results = get_bis(bis, countries, years)
print("4")
wb_results = get_wb(wb, c, y)
print("5")
results = {**eurostat_results, **wb_er_results, **oecd_results, **bis_results, **wb_results}

1
2


/Users/matthewbone/Documents/Coding/VSCode/macro-model-eda/edaEnv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/matthewbone/Documents/Coding/VSCode/macro-model-eda/edaEnv/lib/python3.11/site-packages/data_processing/readers/economic_data/oecd_economic_data.py:99: RuntimeWarning: invalid value encountered in cast
  return col_data.values.astype("int") / self.scale
/Users/matthewbone/Documents/Coding/VSCode/macro-model-eda/edaEnv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/matthewbone/Documents/Coding/VSCode/macro-model-eda/edaEnv/lib/python3.11/site-packages/data_processing/readers/economic_data/oecd_economic_data.py:99: RuntimeWarning: invalid value encountered in cast
  return col_data.values.astype("int") / self.scale
/Users/matthewbone/Documents/Coding/VS

3
4
5


In [60]:
wb_results

{'get_unemployment_rate_worldbank': [[0.0555999994277954,
   0.0469999980926514,
   0.0503999996185303,
   0.0480999994277954,
   0.0459000015258789,
   0.0475,
   0.0534999990463257,
   0.05260000228881841,
   0.0561999988555908,
   0.0753999996185303,
   0.0778999996185303,
   0.0803999996185303,
   0.0788000011444092,
   0.0751999998092651,
   0.061100001335144,
   0.0530000019073486,
   0.0480999994277954,
   0.0432999992370605,
   0.04,
   0.0374000000953674,
   0.0447200012207031,
   0.0452600002288818],
  [0.0659000015258789,
   0.061799998283386205,
   0.0690999984741211,
   0.0767999982833862,
   0.073600001335144,
   0.084399995803833,
   0.0825,
   0.0746000003814697,
   0.0698000001907349,
   0.07909999847412111,
   0.08289999961853028,
   0.071399998664856,
   0.0753999996185303,
   0.08430000305175779,
   0.0852000045776367,
   0.0847999954223633,
   0.0782999992370605,
   0.0709000015258789,
   0.059499998092651406,
   0.053600001335144,
   0.0555000019073486,
   0.06420

In [62]:
results['countries'] = countries
results['years'] = years.tolist()

with open("test_data.json", "w") as f:
    f.write(json.dumps(results))